In [174]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings
import itertools
np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

In [175]:
# read data from file
# train = pd.read_csv("../input/train.csv") 
# test = pd.read_csv("../input/test.csv")
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv")


# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [176]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [177]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [178]:
train = train[(train.isnull().sum(axis=1) <= 15)]

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [179]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [180]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [181]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17
(460, 17)
Index(['PhoneId', 'Brand', 'SIM 2', 'os_name', 'Resolution',
       'Screen to Body Ratio (calculated)', 'Height', 'Pixel Density',
       'Weight', 'Screen Size', 'Capacity', 'Num_cores', 'Sim1',
       'Internal Memory', 'RAM', 'Processor_frequency', 'SIM Slot(s)'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Resolution', 'Screen to Body Ratio (calculated)', 'Height',
       'Pixel Density', 'Weight', 'Screen Size', 'Capacity', 'Internal Memory',
       'RAM', 'Processor_frequency', 'Brand_10.or', 'Brand_Apple',
       'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry', 'Brand_Comio',
       'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee', 'Brand_Google',
       'Brand_HTC', 'Brand_Honor', 'Brand_Huawei', 'Brand_InFocus',
       'Brand_Infinix', 'Brand_Intex', 'Brand_Itel', 'Brand_Jivi',
       'Brand_Karbonn', 'Brand_LG', 'Brand_Lava', 'Br

# Solution

In [182]:
# Last column contains the ratings i.e. the label
print(train_new.shape, test_new.shape)

(341, 88) (119, 87)


In [183]:
# Labels have not been binarized
train_new['Rating'][25:40]

25    4.3
26    4.3
27    4.6
28    4.4
29    4.5
30    3.9
31    4.2
32    4.3
33    4.2
34    4.3
35    4.3
36    4.2
37    4.4
38    4.1
39    3.8
Name: Rating, dtype: float64

### Binarize the target labels

In [184]:
train_new['Rating'] = train_new['Rating'].apply(lambda x: 1 if x>=4 else 0)

In [185]:
train_new['Rating'][25:40]

25    1
26    1
27    1
28    1
29    1
30    0
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    0
Name: Rating, dtype: int64

### Split train_new into X and y.

In [186]:
y = train_new['Rating']
X = train_new.drop('Rating', axis = 1)  # Not inplace, train_new is not affected

In [187]:
print(X.shape, y.shape)

(341, 87) (341,)


In [188]:
# Verifying that y contains the labels. Also data imbalance is observed hence stratified split is necessary in next step
y.value_counts()

1    238
0    103
Name: Rating, dtype: int64

### Split the training data into training and validation set

In [189]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, stratify = y, test_size = 0.1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print(y_train.mean(), y_val.mean())

(306, 87) (35, 87) (306,) (35,)
0.6993464052287581 0.6857142857142857


### Standardizing the data

Standardization should not be done for one hot encoded columns

In [190]:
# Find all columns which are one-hot encoded
not_onehot = list()
onehot = list()
for i in X_train.columns:
    if X_train[i].dtype != 'uint8':
        not_onehot.append(i)
    else:
        onehot.append(i)

Standardizing by **median** instead of mean, makes it **robust to outliers**.

In [191]:
X_train[not_onehot]=(X_train[not_onehot]-X_train[not_onehot].median())/X_train[not_onehot].std()
X_val[not_onehot]=(X_val[not_onehot]-X_val[not_onehot].median())/X_val[not_onehot].std()

In [192]:
X_train.head(10)

,PhoneId,Resolution,Screen to Body Ratio (calculated),Height,Pixel Density,Weight,Screen Size,Capacity,Internal Memory,RAM,...,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,Sim1_2G,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM"
39,-1.275027,0.845493,-0.161122,0.331501,0.849371,0.835420,0.000000,0.990322,0.671006,0.015460,...,0,0,0,0,0,1,0,1,0,0
41,-1.260244,0.000000,1.194382,0.716471,2.321615,1.231146,0.921066,0.463555,0.671006,0.046381,...,0,0,0,0,0,1,0,0,1,0
26,-1.408074,2.705578,0.869019,1.186989,0.747447,0.967329,1.052647,0.779616,2.013018,0.046381,...,0,0,0,0,0,1,0,1,0,0
254,0.905454,0.000000,0.281704,0.320808,-0.498298,0.615573,0.263162,0.990322,0.000000,0.000000,...,0,0,0,0,0,1,0,1,0,0
295,1.341550,-0.507296,-0.316007,0.106936,-0.668172,0.395725,0.000000,-0.063212,0.000000,0.000000,...,0,0,0,0,0,1,0,1,0,0
117,-0.528490,1.352789,-0.034303,0.203178,0.849371,-0.351756,0.000000,0.147495,0.671006,0.015460,...,0,0,0,0,0,1,0,1,0,0
1,-1.681558,0.000000,0.867980,0.513292,-0.622872,0.307786,0.921066,1.232635,0.000000,0.000000,...,0,0,0,0,0,1,0,0,1,0
135,-0.328920,1.352789,0.293138,1.229763,0.464323,1.187176,0.657904,0.990322,0.671006,0.046381,...,0,0,0,0,0,1,0,1,0,0
58,-1.127198,2.874676,1.304568,0.652309,0.872021,0.923359,1.184228,0.463555,2.013018,0.077302,...,0,0,0,0,0,1,0,0,1,0
122,-0.491532,-0.169099,-0.755714,-1.390167,0.000000,-0.571603,-1.052647,-1.305329,0.671006,-0.015460,...,0,0,0,0,0,1,0,0,0,1


In [193]:
train_new.iloc[[39, 41, 26, 254, 295, 117, 1, 135, 58, 122], :]

,PhoneId,Resolution,Screen to Body Ratio (calculated),Height,Pixel Density,Weight,Screen Size,Capacity,Internal Memory,RAM,...,Num_cores_Quad,Num_cores_Tru-Octa,Sim1_2G,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Rating
39,56,13,70.95,154.5,401,180,5.5,4000,64,4,...,0,0,0,0,1,0,1,0,0,0
41,58,8,83.99,158.1,531,189,6.2,3500,64,6,...,0,0,0,0,1,0,0,1,0,1
26,38,24,80.86,162.5,392,183,6.3,3800,128,6,...,0,0,0,0,1,0,1,0,0,1
254,351,8,75.21,154.4,282,175,5.7,4000,32,3,...,0,0,0,0,1,0,1,0,0,1
295,410,5,69.46,152.4,267,170,5.5,3000,32,3,...,0,0,0,0,1,0,1,0,0,0
117,157,16,72.17,153.3,401,153,5.5,3200,64,4,...,0,0,0,0,1,0,1,0,0,1
1,1,8,80.85,156.2,271,168,6.2,4230,32,3,...,0,0,0,0,1,0,0,1,0,1
135,184,16,75.32,162.9,367,188,6.0,4000,64,6,...,0,0,0,0,1,0,1,0,0,1
58,76,25,85.05,157.5,403,182,6.4,3500,128,8,...,0,0,0,0,1,0,0,1,0,1
122,162,7,65.23,138.4,326,148,4.7,1821,64,2,...,0,0,0,0,1,0,0,0,1,1


## Remove unneccessary one-hot columns

Many **one-hot** encoded columns contain **very few positive cases** i.e. value is 1. 

*Hypothesis:* these features will not help the mp neuron model to generalize. As the drawback in mp neuron model is that **all columns/features are given equal weight**.

In [194]:
for col in onehot[:10]:
    temp = X[col].value_counts().values
    try:
        print('0:', temp[0], '; 1:', temp[1])
    except:
        print('0:', temp[0], '; 1: 0')

0: 339 ; 1: 2
0: 322 ; 1: 19
0: 333 ; 1: 8
0: 341 ; 1: 0
0: 336 ; 1: 5
0: 340 ; 1: 1
0: 339 ; 1: 2
0: 341 ; 1: 0
0: 331 ; 1: 10
0: 337 ; 1: 4


In [195]:
PERCENTAGE = 0.005

print('Original shape:', X_train.shape, X_val.shape)

one_hot_drop = list()
for i in X_train.columns:
    if X_train[i].dtype == 'uint8':
        if X_train[i].shape[0]*PERCENTAGE > X_train[i].sum(axis=0):
            one_hot_drop.append(i)

X_train.drop(one_hot_drop, axis = 1, inplace=True)
X_val.drop(one_hot_drop, axis = 1, inplace=True)

# Removing PhoneId as it has no predictive power
X_train.drop('PhoneId', axis = 1, inplace=True)
X_val.drop('PhoneId', axis = 1, inplace=True)

print(f'After removing {PERCENTAGE*100}% columns:', X_train.shape, X_val.shape)

Original shape: (306, 87) (35, 87)
After removing 0.5% columns: (306, 65) (35, 65)


## Perceptron Class

In [196]:
class Perceptron:
  
    def __init__ (self):
        self.w = None
        self.b = None

    def model(self, x):
        return 1 if (np.dot(self.w, x) >= self.b) else 0

    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result)
        return np.array(Y)

    def fit(self, X, Y, epochs = 1, lr = 1):

        self.b = 0
        self.w = np.ones(X.shape[1])
        accuracy = {}
        max_accuracy = 0

        for i in range(epochs):
            for x, y in zip(X, Y):
                y_pred = self.model(x)
                if y == 1 and y_pred == 0:
                    self.w = self.w + lr * x
                    self.b = self.b - lr * 1
                elif y == 0 and y_pred == 1:
                    self.w = self.w - lr * x
                    self.b = self.b + lr * 1 

            accuracy[i] = accuracy_score(self.predict(X), Y)
            if (accuracy[i] > max_accuracy):
                max_accuracy = accuracy[i]
                chkptw = self.w
                chkptb = self.b

        self.w = chkptw
        self.b = chkptb

        print('Max training accuracy:',max_accuracy)
        
        return max_accuracy, (chkptw, chkptb)

## Grid Search on LR and Epochs

In [197]:
val_accs = list()

epochs = [5000, 10000, 20000]
LR = [0.0001, 0.001, 0.01, 0.1]

for i, j in list(itertools.product(epochs, LR)):
    perceptron = Perceptron()
    print('epochs:', i, ' ,LR:', j)
    
    train_score, wt_matrix = perceptron.fit(X_train.values, y_train.values, i, j)
    
    Y_pred_test = perceptron.predict(X_val.values)
    
    val_score = accuracy_score(Y_pred_test, y_val.values)
    print('Max Validation accuracy:', val_score)
    print('-'*50)
    val_accs.append((train_score, val_score, i, j, wt_matrix))
    
max_val = 0
Final_config = None
for i in val_accs:
    if i[1]>max_val:
        Final_config = i
        max_val = i[1]

print(Final_config[:-1])
    

epochs: 5000  ,LR: 0.0001
Max training accuracy: 0.8366013071895425
Max Validation accuracy: 0.6
--------------------------------------------------
epochs: 5000  ,LR: 0.001
Max training accuracy: 0.8529411764705882
Max Validation accuracy: 0.4857142857142857
--------------------------------------------------
epochs: 5000  ,LR: 0.01
Max training accuracy: 0.8660130718954249
Max Validation accuracy: 0.7428571428571429
--------------------------------------------------
epochs: 5000  ,LR: 0.1
Max training accuracy: 0.8627450980392157
Max Validation accuracy: 0.7714285714285715
--------------------------------------------------
epochs: 10000  ,LR: 0.0001
Max training accuracy: 0.8431372549019608
Max Validation accuracy: 0.6285714285714286
--------------------------------------------------
epochs: 10000  ,LR: 0.001
Max training accuracy: 0.8660130718954249
Max Validation accuracy: 0.6
--------------------------------------------------
epochs: 10000  ,LR: 0.01
Max training accuracy: 0.8758169

## Final model

Using the best LR and epoch combination from the grid search, we train the final model.

In [198]:
perceptron = Perceptron()

train_score, wt_matrix = perceptron.fit(X_train.values, y_train.values, 10000, 0.01)

Y_pred_test = perceptron.predict(X_val.values) 
val_score = accuracy_score(Y_pred_test, y_val.values)

print('Max Validation accuracy:', val_score)

Max training accuracy: 0.8758169934640523
Max Validation accuracy: 0.8


## Submission of csv file

Same transformation have to be performed on test set as the ones performed on training set. This is the **preprocessing step** before using our model for prediction.

In [199]:
test_new.head()

,PhoneId,Resolution,Screen to Body Ratio (calculated),Height,Pixel Density,Weight,Screen Size,Capacity,Internal Memory,RAM,...,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,Sim1_2G,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM"
0,3,5,80.85,156.2,271,168,6.20,4230,16,2,...,0,0,0,0,0,1,0,0,1,0
1,11,12,81.60,156.0,402,205,6.20,5000,64,4,...,0,0,0,0,0,1,0,0,1,0
2,13,25,83.84,156.7,409,169,6.30,3500,64,6,...,0,0,0,0,0,1,0,0,1,0
3,16,16,83.84,156.7,409,169,6.30,3500,64,4,...,0,0,0,0,0,1,0,0,1,0
4,19,20,77.43,158.6,403,181,5.99,4000,64,4,...,0,0,0,0,0,1,0,0,1,0


In [200]:
## Submission_dropcol0.005_median    --> LB 0.88571
## drop columns percentage = 0.005, dropped phoneid, epochs = 10K , LR = 0.01, median normalization

# Drop PhoneId in test set as well as the insignificant one hot encoded columns. Percentage to drop must be same.
temp = test_new['PhoneId']
test_new[not_onehot]=(test_new[not_onehot]-test_new[not_onehot].median())/test_new[not_onehot].std()
test_new.drop(one_hot_drop, axis = 1, inplace=True)
test_new.drop('PhoneId', axis = 1, inplace=True)

In [201]:
# Predict using model
test_new_preds = perceptron.predict(test_new.values)

# Create dataframe
submission = pd.DataFrame({'PhoneId':temp, 'Class':test_new_preds})
submission = submission[['PhoneId', 'Class']]
submission.head()

# Write to csv file
submission.to_csv("submission.csv", index=False)